In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv("CSV/train.csv")
df_test = pd.read_csv("CSV/test.csv")

In [4]:
df_train.head()

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [5]:
df_train.drop(["id" , "topic"] , inplace=True , axis = 1)
df_test.drop(["id" , "topic"] , inplace=True , axis = 1)

In [6]:
df_train.head()

,dialogue,summary
0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...
3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...
4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...


In [7]:
df_test.head()

,dialogue,summary
0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...
1,"#Person1#: Ms. Dawson, I need you to take a di...",In order to prevent employees from wasting tim...
2,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for #Person1# abo...
3,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....
4,#Person1#: You're finally here! What took so l...,#Person2# decides to follow #Person1#'s sugges...


In [8]:
print(df_train["dialogue"][0])

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.


In [9]:
print(df_train["summary"][0])

Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking.


In [91]:
import pandas as pd
from datasets import Dataset

# Sample train_df DataFrame
# train_df = pd.DataFrame({
#     "dialouge": ["Input text 1", "Input text 2"],
#     "summary": ["Summary 1", "Summary 2"]
# })

# Convert DataFrame to custom data format
custom_data = [
    {"text": row["dialogue"], "summary": row["summary"]}
    for index, row in df_train.iterrows()
]

# Create a Hugging Face Dataset
custom_dataset = Dataset.from_dict({
    "text": [entry["text"] for entry in custom_data],
    "summary": [entry["summary"] for entry in custom_data]
})

# Split the dataset into training and testing sets
# train_test_split = custom_dataset.train_test_split(test_size=0)

# Access the training and testing datasets
train_dataset = custom_dataset
# test_dataset = train_test_split['test']

# Optionally, print out the first few entries to verify
# print(train_dataset[:5])
# print(test_dataset[:5])


In [90]:
import pandas as pd
from datasets import Dataset

# Sample train_df DataFrame
# train_df = pd.DataFrame({
#     "dialouge": ["Input text 1", "Input text 2"],
#     "summary": ["Summary 1", "Summary 2"]
# })

# Convert DataFrame to custom data format
custom_data = [
    {"text": row["dialogue"], "summary": row["summary"]}
    for index, row in df_test.iterrows()
]

# Create a Hugging Face Dataset
custom_dataset = Dataset.from_dict({
    "text": [entry["text"] for entry in custom_data],
    "summary": [entry["summary"] for entry in custom_data]
})

# Split the dataset into training and testing sets
# train_test_split = custom_dataset.train_test_split(test_size=0)

# Access the training and testing datasets
test_dataset = custom_dataset
# test_dataset = train_test_split['test']

# Optionally, print out the first few entries to verify
# print(train_dataset[:5])
# print(test_dataset[:5])

In [10]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\Raghunandhan G\Desktop\Text Summarization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [92]:
train_tokens = train_dataset.map(preprocess_function, batched=True)
test_tokens = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 1500/1500 [00:00<00:00, 2391.87 examples/s]


In [93]:
print(train_tokens[0])

{'text': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr. Smith's getting a check-up, a

In [94]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [95]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="pt")

In [96]:
import evaluate

rouge = evaluate.load("rouge")

In [97]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [98]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [99]:

training_args = Seq2SeqTrainingArguments(
    output_dir="dialouge_summarization_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokens,
    eval_dataset=test_tokens,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Raghunandhan G\Desktop\Text Summarization\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 122/7790 [42:02<29:10:08, 13.69s/it]